<a href="https://colab.research.google.com/github/shouvikcirca/LLMs/blob/gemini/RAG_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install google-generativeai
! pip install chromadb
! pip install langchain-google-genai
! pip install langchain
! pip install langchain-community

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from dotenv import load_dotenv
import os
from langchain_community.document_loaders.telegram import text_to_docs
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
_= load_dotenv('drive/MyDrive/env')
api_key = os.environ['GEMINI_API_KEY']

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=api_key)

In [ ]:
with open('drive/MyDrive/LLM/synthetic_data_cb.txt') as f:
    data = f.read().split('\n')

In [ ]:
years = range(1960,2021)
qlist = []

for y in years:
     for i in data:
            q = i.replace('2021',str(y))
            q+='cb,{}'.format(y)
            qlist.append(q)

In [ ]:
vectordb = Chroma.from_documents(text_to_docs(qlist), embeddings, persist_directory="drive/MyDrive/LLM/chroma_db")

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
template = """
{context}
{input}
"""

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
response=retrieval_chain.invoke({"input":"Closing balance for 2022 ?"})

In [ ]:
print(response["answer"])

Can you share the year-end balance for 2022?cb,2022
